In [1]:
import timm
import torch
import wandb
import fastai
from fastai.callback.wandb import WandbCallback
from fastai.vision.all import *

In [2]:
timm.list_models('*max*', pretrained=True)

['maxvit_nano_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_tiny_rw_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256']

In [3]:
config = SimpleNamespace(
    batch_size=8,
    img_size=224,
    seed=42,
    pretrained=True,
    model_name="maxvit_rmlp_small_rw_224.sw_in1k", # try with maxvit_nano_rw_256.sw_in1k # regnetx_040 coatnet_bn_0_rw_224.sw_in1k
    epochs=3)

def get_pets(batch_size, img_size, seed):
    "The dog breeds pets datasets"
    dataset_path = untar_data(URLs.PETS)
    files = get_image_files(dataset_path/"images")
    dls = ImageDataLoaders.from_name_re(dataset_path, files, 
                                        pat=r'(^[a-zA-Z]+_*[a-zA-Z]+)', 
                                        valid_pct=0.2, 
                                        seed=seed, 
                                        bs=batch_size,
                                        item_tfms=Resize(img_size)) 
    return dls

def train(config):
    "Train the model using the supplied config"
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
        cbs = [MixedPrecision(), WandbCallback(log='all')]
        learn = vision_learner(dls, config.model_name, metrics=[error_rate, accuracy], 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)

In [4]:
train(config=config)